In [ ]:
#Kalpa

In [ ]:
#Arjun

# LSTM and Glove Embeddings

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, Dropout, Bidirectional, Flatten
from tensorflow.keras.models import Sequential



import re

from gensim.models.word2vec import Word2Vec
import gensim.downloader

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kalya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

train = df_train.drop(columns=['id','keyword','location'])
test =  df_test.drop(columns=['id','keyword','location'])

First, some minor cleaning

In [5]:
def tweet_cleaner(df, feature = 'text', target='target'):
  list_of_tokens = []
  list_of_stems = []
  list_of_sentences = []
  labels = []

  for index, row in df.iterrows():

    labels.append(row[target])

    # only alphanumeric words for now.
    pattern = r'\b\w+\b'

    token_list = [x.lower() for x in re.findall(pattern, row[feature]) if x not in stopwords.words('english')]

    stemmed_tokens = [PorterStemmer().stem(token) for token in token_list]

    list_of_tokens.append(token_list)
    list_of_stems.append(stemmed_tokens)
    list_of_sentences.append(" ".join(token_list))

  return list_of_tokens, list_of_stems,  list_of_sentences, labels


In [6]:
tokens, stems, sentences, labels = tweet_cleaner(train)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(sentences,labels, random_state = 972, stratify=labels)

Ok now I tokenize(turn to numbers), and pad

https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e

may have to revisit padding

#### I nead from tensorflow.keras.preprocessing.text.Tokenizer and pad_sequences ... only works in colab

In [8]:
max_sequence_length = 100  # Adjust as needed
max_words = 5000  # Adjust as needed

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length, padding='post',truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length, padding='post',truncating='post')

NameError: name 'Tokenizer' is not defined

In [ ]:
X_train_padded.shape

Ok now I'm going to just downlaod the glove embeddings

In [ ]:
# Load pretrained GloVe embeddings
embedding_dim = 25  # Adjust based on the chosen GloVe model dimension
embedding_matrix = np.zeros((max_words, embedding_dim))

glove_path = 'glove.twitter.27B.25d.txt'


with open(glove_path, encoding='utf-8') as glove_file:
    for line in glove_file:
        values = line.split()
        word = values[0]
        if word in tokenizer.word_index and tokenizer.word_index[word] < max_words:
            embedding_vector = np.array(values[1:], dtype='float32')
            embedding_matrix[tokenizer.word_index[word]] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(64,return_sequences = False)))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train_padded,
          np.asarray(y_train),
          validation_data = (X_test_padded, np.asarray(y_test)),
          epochs=4,
          batch_size=64)


### Got a 80% test AND Train accuracy. virtually no overfitting, however, very low bias, we need atleast 10% more accuracy. o well... i pickled it

In [ ]:
import pickle

with open('arjun_model.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(model, file)

In [ ]:
#Polina